In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import pickle
import re

In [5]:

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Keerththanan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Keerththanan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Keerththanan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [6]:

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer



In [7]:
stopword = list(stopwords.words('english'))

In [8]:
len(stopword)

179

#### load and preprocessing

In [11]:
TrainSet=pd.read_csv("train.csv")
TestSet=pd.read_csv("test.csv")

In [12]:
TrainSet.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
# get text and label from TrainSet
Traintext=TrainSet['text'].values
Trainlabel=TrainSet['target'].values

In [14]:
Trainlabel.shape

(7613,)

In [16]:
Valt=Traintext[:500]
Valla=Trainlabel[:500]
Traint=Traintext[500:]
Trainla=Trainlabel[500:]

In [61]:
# Preprocessing
def Mypreprocessing(sent):
    sentence=[]
    lem=WordNetLemmatizer()
    for sen in sent:
        sen=sen.lower()
        sen=re.sub(r"http\S+","",sen)
        sen=re.sub(r"[^\w\s]","",sen)
        make=""
        for word in sen.split():
            if word not in stopword:
                word=lem.lemmatize(word)
                make=make+word+" "
        sentence.append(make.strip())
    return sentence

In [70]:
def Mypreprocessing_SingleText(sent):
    lem=WordNetLemmatizer()
    sen=sent.lower()
    sen=re.sub(r"http\S+","",sen)
    sen=re.sub(r"[^\w\s]","",sen)
    make=""
    for word in sen.split():
        if word not in stopword:
            word=lem.lemmatize(word)
            make=make+word+" "
    make=make.strip()
    return make

In [71]:
Traint[0]

'Christian Attacked by Muslims at the Temple Mount after Waving Israeli Flag via Pamela Geller - ... http://t.co/LHBZHWq4B9'

In [72]:
Mypreprocessing_SingleText(Traint[0])

'christian attacked muslim temple mount waving israeli flag via pamela geller'

In [73]:
# dump processing function
processing_file="processing.pkl"
pickle.dump(Mypreprocessing_SingleText,open(processing_file,"wb"))

In [18]:
TraintProcessed=Mypreprocessing(list(Traint))
ValtProcessed=Mypreprocessing(list(Valt))


In [40]:
TraintProcessed[0:100]

['christian attacked muslim temple mount waving israeli flag via pamela geller',
 'envw98 nickcocofree juliedicaro jdabe80 worst person questioning julie attacked guy empathy',
 'kelly osbourne attacked racist donald trump remark latino view',
 'eunice_njoki aiii need chill answer calmly like shes attacked',
 'christian attacked muslim temple mount waving israeli flag via pamela geller',
 'christian attacked muslim temple mount waving israeli flag via pamela geller',
 'christian attacked muslim temple mount waving israeli flag via pamela geller',
 'attacked robotlvl 19 ive earned total 6615434 free satoshis robotcoingame bitcoin freebitcoin',
 'christian attacked muslim temple mount waving israeli flag via pamela geller',
 'christinalavv lindsay_wynn3 saw tweet feel really attacked',
 'christian attacked muslim temple mount waving israeli flag via pamela geller',
 'christian attacked muslim temple mount waving israeli flag via pamela geller',
 'christian attacked muslim temple mount wa

### Deep Learning Model

In [12]:

import tensorflow as tf

In [13]:
# preprocessing data
oov_token='<OOV>'
Tokenizer=tf.keras.preprocessing.text.Tokenizer(oov_token=oov_token)

In [14]:
Tokenizer.fit_on_texts(TraintR)

In [15]:
len(Tokenizer.word_index)

15131

In [16]:
TrainSeq=Tokenizer.texts_to_sequences(TraintR)
ValSeq=Tokenizer.texts_to_sequences(ValtR)

In [17]:
# padding the sequence
Trainpad=tf.keras.preprocessing.sequence.pad_sequences(TrainSeq,padding='post')
Valpad=tf.keras.preprocessing.sequence.pad_sequences(ValSeq,padding='post',maxlen=Trainpad.shape[1])

In [20]:
# create a mode
model=tf.keras.Sequential(
[
    tf.keras.layers.Embedding(len(Tokenizer.word_index)+1,100,input_length=Trainpad.shape[1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100,dropout=0.2,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100,dropout=0.2,return_sequences=False)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(50,activation='relu'),
    tf.keras.layers.Dropout(0.01),
    tf.keras.layers.Dense(20, activation = "relu"),
    tf.keras.layers.Dense(1,activation='sigmoid')
    
])


In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 100)           1513200   
_________________________________________________________________
bidirectional (Bidirectional (None, 25, 200)           160800    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               240800    
_________________________________________________________________
flatten (Flatten)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                10050     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)               

In [22]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [23]:
model.fit(Trainpad,Trainla,validation_data=(Valpad,Valla),epochs=20,batch_size=64,callbacks=tf.keras.callbacks.EarlyStopping(patience=6))

2022-10-24 01:04:20.082523: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-10-24 01:04:27.207168: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


104/104 [==============================] - 14s 28ms/step - loss: 0.5637 - accuracy: 0.6944 - val_loss: 0.4646 - val_accuracy: 0.7780
Epoch 2/20
104/104 [==============================] - 2s 16ms/step - loss: 0.2923 - accuracy: 0.8825 - val_loss: 0.5459 - val_accuracy: 0.7070
Epoch 3/20
104/104 [==============================] - 2s 16ms/step - loss: 0.1554 - accuracy: 0.9460 - val_loss: 0.6018 - val_accuracy: 0.7350
Epoch 4/20
104/104 [==============================] - 2s 20ms/step - loss: 0.0988 - accuracy: 0.9672 - val_loss: 0.7965 - val_accuracy: 0.7070
Epoch 5/20
104/104 [==============================] - 2s 20ms/step - loss: 0.0833 - accuracy: 0.9747 - val_loss: 0.8831 - val_accuracy: 0.7150
Epoch 6/20
104/104 [==============================] - 2s 16ms/step - loss: 0.0691 - accuracy: 0.9779 - val_loss: 0.9028 - val_accuracy: 0.7220
Epoch 7/20
104/104 [==============================] - 2s 16ms/step - loss: 0.0915 - accuracy: 0.9723 - val_loss: 0.8515 - val_accuracy: 0.6970


In [ ]:
model.evaluate(Valpad,Valla)

### Machine Learning Model

In [19]:
# Feature extraction with Bag of Words and TF-IDF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
countvector=CountVectorizer()
tfidfvector=TfidfVectorizer()

In [23]:
tfidfvector.fit(TraintProcessed)
countvector.fit(TraintProcessed)

CountVectorizer()

In [ ]:
tfidfvector.get_feature_names_out()[0:1000]

In [ ]:
countvector.get_feature_names_out()[0:1000]

In [32]:
X_train_TF=tfidfvector.transform(TraintProcessed).toarray()
X_train_BW=countvector.transform(TraintProcessed).toarray()

In [80]:
len(X_train_TF[0])

15942

In [33]:
X_test_TF=tfidfvector.transform(ValtProcessed).toarray()
X_test_BW=countvector.transform(ValtProcessed).toarray()

In [53]:
# save TF-IDF
tf_idf="TF_IDF.pkl"
pickle.dump(tfidfvector,open(tf_idf,"wb"))

In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [46]:
models=[LogisticRegression()]

In [47]:

score_TF=cross_val_score(LogisticRegression(),X_train_TF,Trainla,cv=5)
score_BW=cross_val_score(LogisticRegression(),X_train_BW,Trainla,cv=5)
print("--Score with TF-IDF ",score_TF)
print("--Score with Bag of Words ",score_BW)

--Score with TF-IDF  [0.71820098 0.68446943 0.67884751 0.68565401 0.78691983]
--Score with Bag of Words  [0.71117358 0.66971188 0.66408995 0.68424754 0.76230661]


In [48]:
# grid search for Logistic
grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
LogisticGridModel=GridSearchCV(LogisticRegression(),grid,cv=5)

In [49]:
LogisticGridModel.fit(X_train_TF,Trainla)

d:\Kaggle\NLP-Disaster-Tweets\Env\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\Kaggle\NLP-Disaster-Tweets\Env\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2']})

In [50]:
LogisticGridModel.best_score_

0.7108183519100018

In [51]:
model_name="LogisticModel.pkl"
pickle.dump(LogisticGridModel,open(model_name,"wb"))

#### load and testing the model

In [74]:
p=pickle.load(open("processing.pkl","rb"))
idf=pickle.load(open("TF_IDF.pkl","rb"))
m=pickle.load(open("LogisticModel.pkl","rb"))

'hey'

In [88]:
l=[]
l.append(p("Hey! How are you?"))
m.predict(idf.transform(l).toarray())[0]

0